# Building the design matrix (DARTEL)

The objective of this analysis was to show consistency with results obtained with TBSS by reproducing them using SPM.

In [23]:
%run /home/grg/git/alfa/nilearn-helper.py
%run /home/grg/git/alfa/build_matrix.py
%matplotlib inline

In [28]:
import json
dd = '/home/grg/SPM/2018'
data_dir = '/home/grg/data/AmylStaging/dartel_final/spm/MD/'
subjects_amylneg = json.load(open(osp.join(dd, 'amyloid_negative.json')))
matched_groups = json.load(open(osp.join(dd, 'subjects_groups.json')))
amyloid_subjects = json.load(open(osp.join(dd, 'amyloid_subjects.json')))

failed = [11055, 10779, 11111, 11114, 11540, 12783, 12810]
subjects = sorted(set(amyloid_subjects)) #.difference(set(failed)))


In [29]:
alfa = pd.read_csv('/home/grg/SPM/2018/ALFA PET Amyloid Staging.csv')
centiloids = alfa[['ID','CENTILOIDS']].set_index('ID').to_dict()['CENTILOIDS']

In [35]:
cov = pd.read_excel(osp.join(dd, 'covariates.xlsx'), converters={'subject':int, 'apoe':int})

images = [e for e in sorted(glob(osp.join(data_dir, '*nii'))) if string.atoi(e.split('/')[-1].split('_')[0]) in subjects]
print len(images)
subjects = [string.atoi(osp.basename(str(e))[:5]) for e in images]
data = build_matrix(images, ['age', 'gender', 'apoe', 'centiloids'], cov, subjects=subjects)
data = build_dummy(data, 'apoe')
data = build_interaction(data, 'centiloids', 'apoe')
del data['apoe']
#del data['stage']
data.head()
mat = build_tbss_matrix(data)
print mat

114
/NumWaves 9
/NumPoints 114
/Matrix
63.4223134839151 2.0 -7.62 1.0 0.0 0.0 -7.62 -0.0 -0.0
54.757015742642 1.0 -1.84 1.0 0.0 0.0 -1.84 -0.0 -0.0
50.3901437371663 2.0 -0.16 0.0 0.0 1.0 -0.0 -0.0 -0.16
54.1519507186858 1.0 -4.12 1.0 0.0 0.0 -4.12 -0.0 -0.0
52.8788501026694 1.0 -17.65 1.0 0.0 0.0 -17.65 -0.0 -0.0
67.5537303216975 2.0 9.38 0.0 1.0 0.0 0.0 9.38 0.0
64.8542094455852 2.0 -0.7 1.0 0.0 0.0 -0.7 -0.0 -0.0
64.4052019164956 1.0 -18.08 0.0 1.0 0.0 -0.0 -18.08 -0.0
63.8412046543463 1.0 8.86 1.0 0.0 0.0 8.86 0.0 0.0
60.4736481861739 1.0 -8.48 1.0 0.0 0.0 -8.48 -0.0 -0.0
65.6974674880219 2.0 51.58 0.0 1.0 0.0 0.0 51.58 0.0
66.1382614647502 2.0 69.6 1.0 0.0 0.0 69.6 0.0 0.0
60.8268309377139 1.0 -3.73 1.0 0.0 0.0 -3.73 -0.0 -0.0
51.1403148528405 1.0 -2.52 0.0 0.0 1.0 -0.0 -0.0 -2.52
66.0670773442847 1.0 19.27 1.0 0.0 0.0 19.27 0.0 0.0
54.6255989048597 1.0 18.98 0.0 0.0 1.0 0.0 0.0 18.98
60.6735112936345 1.0 19.42 0.0 1.0 0.0 0.0 19.42 0.0
66.031485284052 1.0 -2.52 1.0 0.0 0.0 -2.52 -

In [34]:
contrasts = tbss_covariates_simple_contrasts(data.columns)
pair_contrasts = tbss_2vs2_contrasts(['centiloids%i'%i for i in range(0,3)], list(data.columns))
contrasts.extend(pair_contrasts)
print build_tbss_contrasts(contrasts)


/NumWaves 10
/ContrastName1 images(+)
/ContrastName2 images(-)
/ContrastName3 age(+)
/ContrastName4 age(-)
/ContrastName5 gender(+)
/ContrastName6 gender(-)
/ContrastName7 centiloids(+)
/ContrastName8 centiloids(-)
/ContrastName9 0(+)
/ContrastName10 0(-)
/ContrastName11 1(+)
/ContrastName12 1(-)
/ContrastName13 2(+)
/ContrastName14 2(-)
/ContrastName15 centiloids0(+)
/ContrastName16 centiloids0(-)
/ContrastName17 centiloids1(+)
/ContrastName18 centiloids1(-)
/ContrastName19 centiloids2(+)
/ContrastName20 centiloids2(-)
/ContrastName21 centiloids0>centiloids1
/ContrastName22 centiloids0>centiloids2
/ContrastName23 centiloids1>centiloids0
/ContrastName24 centiloids1>centiloids2
/ContrastName25 centiloids2>centiloids0
/ContrastName26 centiloids2>centiloids1
/NumContrasts 26
/Matrix
-1 0 0 0 0 0 0 0 0 0
-1 0 0 0 0 0 0 0 0 0
0 -1 0 0 0 0 0 0 0 0
0 -1 0 0 0 0 0 0 0 0
0 0 -1 0 0 0 0 0 0 0
0 0 -1 0 0 0 0 0 0 0
0 0 0 -1 0 0 0 0 0 0
0 0 0 -1 0 0 0 0 0 0
0 0 0 0 -1 0 0 0 0 0
0 0 0 0 -1 0 0 0 0 0